In [2]:
import numpy as np
import re 
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression

def getnumbers_only(input_string): 
'''Used to filter megawatt hours from the html code'''
    match = re.search(r'[\d,]+', input_string)

    #Check if a match was found
    if match:
        #Remove commas from the matched string and convert it to an integer
        number = int(match.group().replace(',', ''))
        return number
    else:
        print("No number found in the input string.")

In [3]:
file_object = open("battery-report.html", "r")
html_data = file_object.read()
#print(data)
file_object.close()
data = BeautifulSoup(html_data, 'html.parser')

In [4]:
target_text = "Battery capacity history"
h2_element = data.find("h2", string=lambda text: text and text.strip() == target_text) 
#addresses the issue of exact matching by stripping the whitespaces and \ns near the target string"
if h2_element:
    #gets everything between <h2>Heading 2A</h2> and <h2>Heading 2B</h2>
    content = []
    #starts from the next element after the <h2>Heading 2A</h2>
    current_element = h2_element.find_next_sibling()
    while current_element and current_element.name != 'h2':
        content.append(str(current_element))
        current_element = current_element.find_next_sibling()
    #joins the content
    result = '\n'.join(content)
    print(result)
else:
    print(f"didnt find with {target_text} sed :()")

<div class="explanation">

      Charge capacity history of the system's batteries

    </div>
<table><colgroup><col/><col class="col2"/><col style="width: 10em;"/></colgroup><thead><tr><td><span>PERIOD</span></td><td class="centered">

            FULL CHARGE CAPACITY

          </td><td class="centered">

            DESIGN CAPACITY

          </td></tr></thead><tr class="even 1"><td class="dateTime">2021-01-04

      - 2021-01-18</td><td class="mw">72,261 mWh

        </td><td class="mw">75,998 mWh

        </td></tr><tr class="odd 2"><td class="dateTime">2021-01-18

      - 2021-01-25</td><td class="mw">72,111 mWh

        </td><td class="mw">75,998 mWh

        </td></tr><tr class="even 3"><td class="dateTime">2021-01-25

      - 2021-02-01</td><td class="mw">72,082 mWh

        </td><td class="mw">75,998 mWh

        </td></tr><tr class="odd 4"><td class="dateTime">2021-02-01

      - 2021-02-16</td><td class="mw">72,082 mWh

        </td><td class="mw">75,998 mWh

        </td><

table = BeautifulSoup(result,"html.parser")
charge_capacity_all = table.find_all("td", class_ = "mw")#contains the other column("Original Charge")

#table.find_all("td", class_ = "dateTime") the dates are in week times and then rest in single days(not a full week)
if charge_capacity_all:
    charge_capacity_new = charge_capacity_all[::2]#with the other column removed

    charge_capacity_array = []
    
    for charge in charge_capacity_new:
        mwh = charge.prettify()
        charge_capacity_array.append(mwh)

else: 
    print("not found the mw class... please double check")

charge_capacity_array
mwh

In [ ]:
dates = table.find_all("td", class_ = "dateTime") 

NameError: name 'table' is not defined